In [ ]:
import pandas as pd
import geopy
import folium
import json
from PIL import Image
from google.colab import files

uploaded = files.upload()

In [ ]:
df = pd.read_excel('Universities.xlsx', 'Data')
df = df.apply(lambda x: x.str.lower() if x.dtype == 'object' else x)
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)
df.dropna(how='all', axis=1, inplace=True)

In [ ]:
# University coordinates for map
geolocator = geopy.Nominatim(user_agent='example')

def get_latitude(address):
    location = geolocator.geocode(address)
    if location:
        return location.latitude
    else:
        return None
    
    
def get_longitude(address):
    location = geolocator.geocode(address)
    if location:
        return location.longitude
    else:
        return None
    
df['Latitude'] = df['University'].apply(get_latitude)
df['Longitude'] = df['University'].apply(get_longitude)

In [ ]:
def ratio_creator(image):
    """ Returns tuple of dimensions of resized image. """
    image = Image.open(image)
    width, height = image.size
    ratio = height / width
    if ratio < 0.25:
        new_height = 80 * ratio
        new_width = 80
    elif ratio < 0.75:
        new_height = 60 * ratio
        new_width = 60
    elif ratio < 1.25:
        new_height = 40 * ratio
        new_width = 40
    else:
        new_height = 20 * ratio
        new_width = 20
    return (round(new_width), round(new_height))

In [ ]:
# Map of universities (focused on California)
coordinates = df[['University', 'Latitude', 'Longitude']].drop_duplicates().sort_values('Longitude')
        
california_map = folium.Map(location=[36.7783, -119.4179], 
                            tiles='https://server.arcgisonline.com/ArcGIS/rest/services/Canvas/World_Light_Gray_Base/MapServer/tile/{z}/{y}/{x}',
                            attr='Tiles &copy; Esri &mdash; Esri, DeLorme, NAVTEQ',
                            zoom_start=6)

with open('us states geojson.json') as handle:
    country_geo = json.loads(handle.read())
    
for i in country_geo['features']:
    if i['properties']['NAME'] == 'California':
        country = i
        break
        
folium.features.Choropleth(geo_data=country, fill_color='#addfff', line_weight=.25, control=False).add_to(california_map)

pointers = folium.FeatureGroup(name='Pointers')
logos = folium.FeatureGroup(name='Logos')
california_map.add_child(pointers)
california_map.add_child(logos)

for i, row in coordinates.iterrows():
    try:
        icon = folium.features.CustomIcon(icon_image=f"{row.University}.png", icon_size=ratio_creator(f"{row.University}.png"))
        line_icon = folium.features.CustomIcon(icon_image="polyline.png", icon_size=(50, 50))
        folium.Marker(location=[row.Latitude, row.Longitude], icon=icon, draggable=True).add_to(logos)
        folium.Marker(location=[row.Latitude, row.Longitude], icon=line_icon, draggable=True).add_to(pointers)
        folium.CircleMarker((row.Latitude, row.Longitude), radius=2, opacity=1, weight=2, color='blue', fill='blue').add_to(california_map)
    except FileNotFoundError:
        continue

# When dragging items on map, maintain a fixed zoom level and toggle pointers or logos from the layer control 
# on top right to make it easier
# Tip from the developer: zoom in to the map, grab a logo or pointer, then zoom out to the desired level and then 
# place the item in the proper place
folium.LayerControl().add_to(california_map)

# Open saved html file to make adjustments and take final screenshot (larger image)
# california_map.save('cal_map.html')
california_map